For the final project, I'll analyze the CoffeeKing dataset (Yelp Academic Dataset), to see if there's anything interesting we can dig out.  
Here's the discription:  
CoffeeKing is a new startup coffee company providing a unique and novel experience to their customers.  They want to appeal to a wide variety of clientele.  You will use Yelp reviews and user data to provide insights to CoffeeKing for things such as location selection and/or hours of operation (and other questions?)  
  
### First, import the packages we needed to do EDA.

In [138]:
import pandas as pd
import numpy as np
from pandasql import sqldf
# import importlib
# importlib.reload(util)
from util import *

Check for data's basic information.

In [102]:
# Check for total size
json_path = 'C:/Users/user/Desktop/CoffeKing (Yelp Academic Dataset)/'

print('size of store: ', count_total_rows(json_path, 'yelp_academic_dataset_business.json', 100000))
print('size of review: ', count_total_rows(json_path, 'yelp_academic_dataset_review.json', 100000))
print('size of user: ', count_total_rows(json_path, 'yelp_academic_dataset_user.json', 100000))

size of store:  160585
size of review:  8635403
size of user:  2189457


In [152]:
# Find stores selling coffee
yelp_bs = pd.read_json(json_path + 'yelp_academic_dataset_business.json', lines=True)
yelp_bs['categories'] = yelp_bs['categories'].map(lambda x: to_lower_case(x))
yelp_bs = yelp_bs[yelp_bs['categories'].str.contains('coffee').map(lambda x: True if x is True else False)] # Find store category with "coffee"
coffee_id = list(yelp_bs['business_id'])

In [154]:
# Deal with review, append reviews of the coffee stores
coffee_review = []
with open(json_path + "yelp_academic_dataset_review.json", "r") as f:
    reader = pd.read_json(json_path + "yelp_academic_dataset_review.json", 
                        lines=True, chunksize=100000)
    for chunk in reader:
        chunk = chunk[chunk['business_id'].isin(coffee_id)]
        coffee_review.append(chunk)
f.close()

In [155]:
yelp_rv = pd.concat(coffee_review, ignore_index=True)
user_id = yelp_rv['user_id'].unique().tolist()

In [156]:
# Search for user information
user_info = []
with open(json_path + "yelp_academic_dataset_user.json", "r") as f:
    reader = pd.read_json(json_path + "yelp_academic_dataset_user.json", 
                        lines=True, chunksize=100000)
    for chunk in reader:
        chunk = chunk[chunk['user_id'].isin(user_id)]
        user_info.append(chunk)
f.close()

In [157]:
yelp_user = pd.concat(user_info, ignore_index=True)

In [158]:
# Drop columns we don't need
yelp_bs.drop(["address", "postal_code", "latitude", "longitude"], axis=1, inplace=True)
yelp_rv.drop(["review_id"], axis=1, inplace=True)
yelp_user.drop(["name", "friends"], axis=1, inplace=True)

In [159]:
# Change column names that may duplicated
yelp_bs.rename(columns={"stars":"store_stars", "review_count":"store_review_count"}, inplace=True)
yelp_rv.rename(columns={"stars":"rv_stars", "useful":"rv_useful", "funny":"rv_funny", "cool":"rv_cool"}, inplace=True)
yelp_user.rename(columns={"review_count":"user_review_count"}, inplace=True)

In [160]:
# Merge tables, base on review
yelp_coffee = pd.merge(pd.merge(yelp_rv, yelp_bs, on="business_id", how='left'), 
                        yelp_user, on="user_id", how='left')

In [162]:
# Save merged table
yelp_coffee.to_csv(json_path + 'yelp_coffee.csv')

In [161]:
# Brief descrive of our merged table
print('We have {} distinct coffee stores.'.format(yelp_coffee['business_id'].unique().shape[0]))
print('With {} reviews about those stores.'.format(yelp_coffee.shape[0]))
print('And {} distinct users comment on them.'.format(yelp_coffee['user_id'].unique().shape[0]))
print('Our merged table has {} rows and {} features.'.format(yelp_coffee.shape[0],yelp_coffee.shape[1]))

We have 7747 distinct coffee stores.
With 605303 reviews about those stores.
And 321890 distinct users comment on them.
Our merged table has 605303 rows and 36 features.


### Now we can start to find what's in our table.

In [163]:
yelp_coffee

,user_id,business_id,rv_stars,rv_useful,rv_funny,rv_cool,text,date,name,city,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,JHXQEayrDHOWGexs0dCviA,KXCXaF5qimmtKKqnPc_LQA,1,0,0,0,Great coffee and pastries. Baristas are excell...,2018-03-03 23:45:25,Thierry,Vancouver,...,2,0,0,0,2,11,2,2,0,1
1,LV1ME-ibA2h0IGyFUUWhaQ,H_RM2u1WWGU1HkKZrYq2Ow,5,0,0,0,Incredible donuts. Sometimes you have to go ea...,2017-08-07 19:34:13,Blue Star Donuts,Portland,...,0,0,0,0,0,1,0,0,0,0
2,JuM-lH05m6Ln8OPUTg8p0g,H_RM2u1WWGU1HkKZrYq2Ow,5,0,0,0,"Dont bother going to voodoo, just come here in...",2016-09-18 17:06:01,Blue Star Donuts,Portland,...,0,0,0,0,0,1,0,0,0,0
3,4cDqW9sWnZ57tsIEiElJWw,0nMbLN0B5I-2tNnQLSo86w,5,4,4,4,"""Even the mad Captain Ahab is touched by Starb...",2017-12-02 19:41:59,Starbucks,Portland,...,16,0,3,0,82,224,237,237,66,75
4,7mWnNVk2n99JxkvV3PW0nA,Un6u2cECyV4nZb_HGZ-uTA,4,1,0,1,It's crazy how establishments on the west coas...,2011-02-13 16:38:09,Tin Shed Garden Cafe,Portland,...,4,1,0,3,12,28,28,28,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605298,im-MIiI-UFJsdrmIaxer2A,edlA6hSakvNxRZQFbuf-Iw,3,0,0,0,I've always gone to sharetea wherever I go for...,2021-01-25 23:04:35,Sharetea,Beaverton,...,0,0,0,0,0,0,0,0,0,0
605299,Cs5isLcwZgK-Cd1SA4TShw,p4sp8NYrFW7MmBajumBrtg,1,1,0,0,Every time I come to this dunkin they don't gi...,2020-06-30 14:21:53,Dunkin',Dunwoody,...,0,0,0,0,0,0,0,0,0,0
605300,_ailSgaI-3ztQxRnZtcluw,3uyvi_du7-sMxkYpcAg0-A,5,0,0,0,This place is poppin'! Very much enjoyed their...,2020-12-13 17:33:09,"Greystone Cafe, Bakery, and Provisions",Boston,...,0,0,0,0,0,0,0,0,1,0
605301,VEB7udH-qIN_ON1SMyWrPg,zM98ZSIJyuBQabyYornLpw,3,0,0,0,Bartender was friendly and attentive. We came ...,2021-01-10 04:10:45,Casa Chapala Mexican Cuisine & Tequila Bar,Austin,...,0,0,0,0,0,0,0,0,0,0
